# Controlo da rota de 3 navios num lago infinito

In [3]:
from z3 import *
import numpy as np
from math import cos, sin

In [4]:
VEL_INIT, VEL_HIGH, VEL_LOW = 0, 10, 1 # velocidade em m/s
THETA = 15 # angulo de viragem em graus
TAU = 1 # intervalo de tempo entre estados temporais consecutivos em ??? (unidade a determinar)
SAFE_DIST = 50 # distancia de seguranca entre barcos em m

GAUSS_DP = 80 # desvio padrao da gaussiana da inicializacao da posicao em m

In [15]:
def declare(i):
    mode = {
        "b1": {"v": Int(f"tr{i}_b1_v"), "a": Int(f"tr{i}_b1_a")},
        "b2": {"v": Int(f"tr{i}_b2_v"), "a": Int(f"tr{i}_b2_a")},
        "b3": {"v": Int(f"tr{i}_b3_v"), "a": Int(f"tr{i}_b3_a")}
    }
    
    state = {
        "b1": {"x": Real(f"tr{i}_b1_x"), "y": Real(f"tr{i}_b1_y")},
        "b2": {"x": Real(f"tr{i}_b2_y"), "y": Real(f"tr{i}_b2_y")},
        "b3": {"x": Real(f"tr{i}_b3_y"), "y": Real(f"tr{i}_b3_y")}
    }
    
    trace = {
        "mode": mode,
        "state": state,
        "time": Int(f"tr{i}_t")
    }
    
    return trace


def init(trace):
    conds = []
    num_boats = len(trace["mode"])
    
    # Generate random angles
    angles = [i*THETA for i in range(int(360/15))]
    np.random.shuffle(angles)
    random_angles = [angles[0] for _ in range(num_boats)]
    
    # Define initial mode
    for i, b in enumerate(trace["mode"]):
        conds.append(trace["mode"][b]["v"] == VEL_HIGH)
        conds.append(trace["mode"][b]["a"] == random_angles[i])
    
    # Randomly defined position according to gaussian
    for i in range(len(trace["state"])):
        conds.append(trace["state"][f"b{i+1}"]["x"] == GAUSS_DP * np.random.randn())
        conds.append(trace["state"][f"b{i+1}"]["y"] == GAUSS_DP * np.random.randn())
    
    # Set initial time to 0
    conds.append(trace["time"] == 0)
    
    return And(conds)


def Abs(x):
    return If(x >= 0, x, -x)


def switch_no_collision(trace, boat_id1, boat_id2):
    cond_x = Abs(trace["state"][f"b{boat_id1}"]["x"] - trace["state"][f"b{boat_id2}"]["x"]) <= SAFE_DIST
    cond_y = Abs(trace["state"][f"b{boat_id1}"]["y"] - trace["state"][f"b{boat_id2}"]["y"]) <= SAFE_DIST
    
    return And(cond_x, cond_y)


def untimed_to_imminent(trace_ant, trace_atual, boat_id1, boat_id2):
    # Criar o novo estado atual
    num_boats = len(trace_ant["mode"])
    boat_ids = [i for i in range(1, num_boats+1) if (i!=boat_id1 and i!=boat_id2)]
    
    # Condição z3 de colisão iminente
    conds = []
    conds.append(Not(switch_no_collision(trace_ant, boat_id1, boat_id2)))
    
    # Condição z3 do modo
    conds.append(trace_atual["mode"][f"b{boat_id1}"]["v"] == VEL_LOW)
    conds.append(trace_atual["mode"][f"b{boat_id2}"]["v"] == VEL_LOW)
    conds.append(trace_atual["mode"][f"b{boat_id1}"]["a"] == trace_ant["mode"][f"b{boat_id1}"]["a"] + THETA)
    conds.append(trace_atual["mode"][f"b{boat_id2}"]["a"] == trace_ant["mode"][f"b{boat_id2}"]["a"] + THETA)
    for boat_id in boat_ids:
        conds.append(trace_atual["mode"][f"b{boat_id}"]["v"] == trace_ant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_atual["mode"][f"b{boat_id}"]["a"] == trace_ant["mode"][f"b{boat_id}"]["a"])
    
    # Condição z3 do estado
    for boat_id in range(1, num_boats+1):
        conds.append(trace_atual["state"][f"b{boat_id}"]["x"] == trace_ant["state"][f"b{boat_id}"]["x"])
        conds.append(trace_atual["state"][f"b{boat_id}"]["y"] == trace_ant["state"][f"b{boat_id}"]["y"])
    conds.append(trace_atual["time"] == trace_ant["time"])
    
    return And(conds)


def untimed_to_safe(trace_ant, trace_atual, boat_id1):
    # Criar o novo estado atual
    num_boats = len(trace_ant["mode"])
    
    # Condicao z3 do barco estar a salvo
    conds = []
    boat_ids = [i for i in range(1, num_boats+1) if i!=boat_id1]
    conds.append(And([switch_no_collision(trace_ant, boat_id1, boat_id) for boat_id in boat_ids]))
    
    # Condicao z3 do modo
    conds.append(trace_atual["mode"][f"b{boat_id1}"]["v"] == VEL_HIGH)
    conds.append(trace_atual["mode"][f"b{boat_id1}"]["a"] == trace_ant["mode"][f"b{boat_id1}"]["a"])
    for boat_id in boat_ids:
        conds.append(trace_atual["mode"][f"b{boat_id}"]["v"] == trace_ant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_atual["mode"][f"b{boat_id}"]["a"] == trace_ant["mode"][f"b{boat_id}"]["a"])
    
    # Condicao z3 do estado
    for boat_id in range(1, num_boats+1):
        conds.append(trace_atual["state"][f"b{boat_id}"]["x"] == trace_ant["state"][f"b{boat_id}"]["x"])
        conds.append(trace_atual["state"][f"b{boat_id}"]["y"] == trace_ant["state"][f"b{boat_id}"]["y"])
    conds.append(trace_atual["time"] == trace_ant["time"])
    
    return And(conds)         
    

def timed(trace_ant, trace_atual):
    # Definir evolucao do tempo
    conds = []
    num_boats = len(trace_ant["mode"])
    conds.append(trace_atual["time"] == trace_ant["time"] + TAU)
    
    # Definir evolucao do estado
    for boat_id in range(1, num_boats+1):
        # Definir evolucao do modo do barco boat_id
        conds.append(trace_atual["mode"][f"b{boat_id}"]["v"] == trace_ant["mode"][f"b{boat_id}"]["v"])
        conds.append(trace_atual["mode"][f"b{boat_id}"]["a"] == trace_ant["mode"][f"b{boat_id}"]["a"])
        
        # Calculo do incremento da posicao
        dx = trace_ant["mode"][f"b{boat_id}"]["v"] * cos(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
        dy = trace_ant["mode"][f"b{boat_id}"]["v"] * sin(trace_ant["mode"][f"b{boat_id}"]["a"]) * TAU
        
        # Condicao z3 da posicao
        conds.append(trace_atual["state"][f"b{boat_id}"]["x"] == trace_ant["state"][f"b{boat_id}"]["x"] + dx)
        conds.append(trace_atual["state"][f"b{boat_id}"]["y"] == trace_ant["state"][f"b{boat_id}"]["y"] + dy)
    
    return And(cond_t, cond_x, cond_y)


def trans(trace_ant, trace_atual):
    num_boats = len(trace_ant["mode"])
    
    # Untimed transitions
    untimed_conds = []
    
    for boat_id in range(1, num_boats+1): # Untimed to safe
        untimed_conds.append(untimed_to_safe(trace_ant, trace_atual, boat_id))
        
    for boat_id1 in range(1, num_boats+1): # Untimed to imminent
        for boat_id2 in range(boat_id1+1, num_boats+1):
            untimed_conds.append(untimed_to_imminent(trace_ant, trace_atual, boat_id1, boat_id2))
            
    untimed_cond = Or(untimed_conds)
            
    # Timed transitions
    timed_conds = timed(trace_ant, trace_atual)
    
    return Or(untimed_conds, timed_conds)

In [16]:
def time_evolution(declare, init, trans, k):
    # Criar o solver e o traço de estados temporais
    solver = Solver()
    trace = {i: declare(i) for i in range(k)}

    # Inicializar o primeiro estado temporal
    solver.add(init(trace[0]))

    # Fazer a transição de estados
    for i in range(1, k):
        solver.add(trans(trace[i-1], trace[i]))

    if solver.check() == sat:
        m = solver.model()
        print(m)
        r = m
    else:
        r = None
        
    return r

time_evolution(declare, init, trans, 10)

TypeError: must be real number, not ArithRef